# Azure OpenAI - AlphaCentauri Project
## Step 1: Setting up your Azure & Langchain
based on https://learn.deeplearning.ai/courses/building-your-own-database-agent

In [1]:
import os
import pandas as pd
from IPython.display import Markdown, HTML, display
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)


True

In [2]:

from openai import AzureOpenAI

# Azure OpenAI Configuration (CORRECT endpoint from Azure Portal)
#endpoint = "https://js-alphacentauri-resource.cognitiveservices.azure.com/"
endpoint ="https://js-alphacentauri-resource.openai.azure.com/"
deployment = "gpt-4.1-mini"
v_model = "gpt-4.1-mini"
api_version = "2024-12-01-preview"

# Get API key from environment
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")

# Create Azure OpenAI client
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

print("✅ Azure OpenAI client configured")
print(f"Endpoint: {endpoint}")
print(f"Deployment: {deployment}")
print(f"API Version: {api_version}")


✅ Azure OpenAI client configured
Endpoint: https://js-alphacentauri-resource.openai.azure.com/
Deployment: gpt-4.1-mini
API Version: 2024-12-01-preview


In [3]:
# Test the connection
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "Say hello and tell me you're working! Tell me not to worry, everything will be fine.",
        }
    ],
    model=deployment
)

print(response.choices[0].message.content)


Hello! I'm here and working hard for you. Don't worry, everything will be fine! If you need anything, just let me know.


### LangChain Integration


In [4]:
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

# Create LangChain model
model = AzureChatOpenAI(
    openai_api_version=api_version,
    azure_deployment=deployment,
    azure_endpoint=endpoint,
    api_key=subscription_key
)

# Test with LangChain
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Translate 'Hello, how are you?' to French and Spanish.")
]

response = model.invoke(messages)
print(response.content)


Sure! 

- French: Bonjour, comment ça va ?  
- Spanish: Hola, ¿cómo estás?


### Streaming Response Example


In [5]:
v_messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Translate this sentence from English "
    "to French and Spanish. I like red cars and "
    "blue houses, but my dog is yellow.")
]   

## alternative way to stream the response
    # messages=[
    #     {
    #         "role": "system",
    #         "content": "You are a helpful assistant.",
    #     },
    #     {
    #         "role": "user",
    #         "content": "Translate this sentence from English "
    # "to French and Spanish. I like red cars and "
    # "blue houses, but my dog is yellow.",
    #     }
    # ],

# Stream the output for better user experience
response = client.chat.completions.create(
    stream=True,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "Translate this sentence from English "
    "to French, West Vlaams and Spanish. I like red cars and "
    "blue houses, but my dog is yellow.",
        }
    ],
    model=deployment,
)

print("Streaming response:")
for update in response:
    if update.choices:
        print(update.choices[0].delta.content or "", end="")


Streaming response:
Sure! Here is the translation of the sentence "I like red cars and blue houses, but my dog is yellow." into French, West Vlaams, and Spanish:

French:  
J'aime les voitures rouges et les maisons bleues, mais mon chien est jaune.

West Vlaams:  
Ik zin gek op roode wagens en blauwe huzen, moar m'n hoe is geel.

Spanish:  
Me gustan los coches rojos y las casas azules, pero mi perro es amarillo.

### 🔍 Debug: Verify Deployment Name


In [6]:
model.invoke((v_messages))

AIMessage(content="Sure! Here is the translation:\n\nFrench: J'aime les voitures rouges et les maisons bleues, mais mon chien est jaune.  \nSpanish: Me gustan los coches rojos y las casas azules, pero mi perro es amarillo.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 41, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CWrXNOEGK2C0gu2L7mQUCFMoVU0fB', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, '

## Step 2: Interacting with CSV Data

In [7]:
# Create LangChain model
model = AzureChatOpenAI(
    openai_api_version=api_version,
    azure_deployment=deployment,
    azure_endpoint=endpoint,
    api_key=subscription_key
)


### Load the dataset
- baseline model -->
- could use 
  a) fine tuning for sql tasks 
  b) RAG (database as a source)
    - use Langchain agents to connect to SQL Database or CSV file
    - via Azure OpenAI Assistants API (function calling + code interpreter) (stateful management + short term memory)
    - via Azure OpenAI Fucntion Calling: to perform tasks based on your questions
    - Vai Native Database API

In [8]:
#read in the data from the csv file
#df = pd.read_csv("./data/synthetic_sales_data.csv").fillna(value = 0)
df = pd.read_csv("./data/synthetic_sales_data.csv", sep=";").fillna(value=0)

In [9]:
# Check if the file is read and display the first few rows with headers
print("First 5 rows of the dataset:")
print("=" * 80)
display(df.head())

# Alternative: use this if display() doesn't work
# print(df.head().to_string())

First 5 rows of the dataset:


,Region,ProductLine,Month,Revenue,UnitsSold,DiscountPercent,Cost,Profit,ProfitMargin,KPI_01,...,KPI_31,KPI_32,KPI_33,KPI_34,KPI_35,KPI_36,KPI_37,KPI_38,KPI_39,KPI_40
0,East,Clothing,2023-01-01 00:00:00,181890,69,"0,28","122553,98","59336,02","0,33","104,4",...,"100,97","86,57","118,95","95,2","104,3","116,46","105,26","82,91","117,61","90,59"
1,East,Clothing,2023-02-01 00:00:00,316208,952,"0,15","205913,08","110294,92","0,35","107,2",...,"83,42","95,96","83,36","112,95","109,02","127,5","96,35","112,5","105,15","114,32"
2,East,Clothing,2023-04-01 00:00:00,483125,1021,"0,17","400566,74","82558,26","0,17","102,56",...,"112,17","99,13","71,95","106,23","71,27","108,21","121,33","101,07","88,5","75,74"
3,East,Clothing,2023-10-01 00:00:00,476960,1856,"0,13","311779,56","165180,44","0,35","57,08",...,"110,58","124,64","92,32","100,18","108,17","88,18","123,39","101,25","84,87","65,21"
4,East,Electronics,2023-04-01 00:00:00,432515,177,"0,02","301553,9","130961,1","0,3","128,72",...,"92,89","90,96","109,71","90,64","97,44","81,56","89,29","113,26","124,89","96,62"


In [10]:
# Show detailed information about the dataset structure
print("📊 Dataset Information:")
print("-" * 80)
print(f"Total Rows: {len(df)}")
print(f"Total Columns: {len(df.columns)}")
print("\nColumn Names and Data Types:")
print("-" * 80)
for i, (col, dtype) in enumerate(zip(df.columns, df.dtypes), 1):
    print(f"{i:2d}. {col:30s} | Type: {dtype}")
print("-" * 80)


📊 Dataset Information:
--------------------------------------------------------------------------------
Total Rows: 100
Total Columns: 49

Column Names and Data Types:
--------------------------------------------------------------------------------
 1. Region                         | Type: object
 2. ProductLine                    | Type: object
 3. Month                          | Type: object
 4. Revenue                        | Type: int64
 5. UnitsSold                      | Type: int64
 6. DiscountPercent                | Type: object
 7. Cost                           | Type: object
 8. Profit                         | Type: object
 9. ProfitMargin                   | Type: object
10. KPI_01                         | Type: object
11. KPI_02                         | Type: object
12. KPI_03                         | Type: object
13. KPI_04                         | Type: object
14. KPI_05                         | Type: object
15. KPI_06                         | Type: object
16.

#### <span style="color: red">SOME ATTENTION REQUIRED</span> 
create_pandas_dataframe_agent() maak een “agent” die:
- Je dataframe (df) kent
- Je prompt interpreteert (“How many rows are there?”)
- Daarvoor zelf Python-code schrijft
- En die code uitvoert in een “Python REPL” (read–eval–print-loop) in jouw proces.
De “REPL” is in feite een mini-Python-console binnen jouw proces.
En de LLM (of een kwaadaardige prompt) kan daarin eender welke code uitvoeren.
Niet alleen veilige dingen zoals len(df), maar ook bijvoorbeeld:

--> gezien we controle hebben over de data hier kunnen we **allow_dangerous_code=True** toevoegen, MAAR GEBRUIK DIT NIET IN PRODUCTIE!


In [11]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(
    llm=model,
    df=df,
    verbose=True, 
    allow_dangerous_code=True,
    max_iterations=30,  # Increase from default 15
    max_execution_time=300  # 5 minutes timeout
)

agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to find the number of rows in the dataframe `df`.
Action: [python_repl_ast]
Action Input: len(df)[python_repl_ast] is not a valid tool, try one of [python_repl_ast].Question: how many rows are there?
Thought: I need to find the number of rows in the dataframe `df`.
Action: python_repl_ast
Action Input: df.shape[0]100Thought: The dataframe has 100 rows based on the shape output.  
Final Answer: There are 100 rows in the dataframe `df`.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 100 rows in the dataframe `df`.'}

In [12]:
# agent.invoke("What can you tell me about the data?")
# agent.invoke("List me all the diffrent productlines and their individual summed uprevenue")
agent.invoke("List per region which productline sold most units in total and how many units were sold.")




> Entering new AgentExecutor chain...
Thought: I need to group the data by Region and ProductLine and then sum the UnitsSold for each group. After that, I should find the product line with the maximum units sold for each region.

Action: python_repl_ast
Action Input: df.groupby(['Region', 'ProductLine'])['UnitsSold'].sum().reset_index().sort_values(['Region', 'UnitsSold'], ascending=[True, False])   Region  ProductLine  UnitsSold
2    East    Furniture       7533
3    East    Groceries       4459
0    East     Clothing       3898
4    East         Toys       3518
1    East  Electronics       3515
9   North         Toys       5778
5   North     Clothing       4183
6   North  Electronics       4101
7   North    Furniture       3708
8   North    Groceries       2845
13  South    Groceries       9485
11  South  Electronics       7327
10  South     Clothing       6538
14  South         Toys       4535
12  South    Furniture       2664
19   West         Toys       9022
15   West     Clothi

{'input': 'List per region which productline sold most units in total and how many units were sold.',
 'output': '- East: Furniture sold the most units with a total of 7,533 units sold.\n- North: Toys sold the most units with a total of 5,778 units sold.\n- South: Groceries sold the most units with a total of 9,485 units sold.\n- West: Toys sold the most units with a total of 9,022 units sold.'}

In [13]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

#QUESTION = "How may patients were hospitalized during July 2020" 
#"in Texas, and nationwide as the total of all states?"
SIMPLE_PROMPT = """
How many patients were hospitalized during July 2020 in Texas, and nationwide as the total of all states?
Use the hospitalizedIncrease column.

Instructions:
1. Filter the data for July 2020 (dates between 2020-07-01 and 2020-07-31)
2. Sum the hospitalizedIncrease column for Texas
3. Sum the hospitalizedIncrease column for all states
4. Provide both answers with a clear explanation
"""

SIMPLE_PROMPT_SALES = """
How many toys units are sold in the month of July, regardless the year?
"""

QUESTION = "How may patients were hospitalized during July 2020" \
"in Texas, and nationwide as the total of all states?" \
"Use the hospitalizedIncrease column" 


output = agent.invoke(CSV_PROMPT_PREFIX + SIMPLE_PROMPT_SALES + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Question: How many toys units are sold in the month of July, regardless the year?

Thought: First, I need to see all the columns to confirm the relevant column names, then filter the dataset for "ProductLine" == "Toys" and for the month of July in the "Month" column. I will sum the "UnitsSold" values for this filtered data. Then I will verify using another method to be sure.

Action: python_repl_ast
Action Input: |
  import pandas as pd
  pd.set_option('display.max_columns', None)
  df.columns.tolist()SyntaxError: invalid syntax (<unknown>, line 1)Thought: The SyntaxError occurred because the block contains multiple statements but it was passed as a single line input. I should run the code one statement at a time or join them properly in the action input.

I will first set pandas option to display all columns then get the column names.

Action: python_repl_ast
Action Input: |
  import pandas as pd
  pd.set_option('display.max_columns', None)
Acti

In [14]:
# Alternative: Use the complex prompt with reflection (now with increased iteration limit)
# Uncomment the line below to try the more thorough approach with validation
# agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)


In [15]:
# Access the output from the dictionary
print(output['output'])

# Or display it in markdown for better formatting
# display(Markdown(output['output']))

### Number of Toys Units Sold in July (Any Year): **6258**

---

### Explanation:
To find the number of Toys units sold in July, I used the following approach:

- The relevant columns for this query are `"ProductLine"`, `"Month"`, and `"UnitsSold"`.
- Since the `"Month"` column was represented as a string (e.g., `"2023-07-01 00:00:00"`), I extracted the substring corresponding to the month using `df['Month'].str[5:7]`.
- I filtered the DataFrame for rows where `ProductLine == "Toys"` and the extracted month substring was `"07"` (July).
- Then I summed the `"UnitsSold"` column for these filtered rows.

I confirmed the result by applying the filtering in two ways:
1. Directly in a single filtering step with string slicing.
2. Filtering the DataFrame by `ProductLine == 'Toys'` first, then filtering for July using the month substring, finally summing `"UnitsSold"`.

Both methods yielded the same total: **6258** units sold.

This consistent result gives confidence that 6258 is the correct t

## Step 3: Connecting to a SQL Database

In [16]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase


In [17]:
import urllib.request

# Create data directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# Download the file using Python (works on Windows, Mac, Linux)
# url = "https://covidtracking.com/data/download/all-states-history.csv"
# file_path = "./data/all-states-history.csv"

# print(f"Downloading from {url}...")
# urllib.request.urlretrieve(url, file_path)
# print(f"✅ File downloaded to {file_path}")

# # Load the data
# df = pd.read_csv(file_path).fillna(value=0)
# print(f"✅ Loaded {len(df)} rows of data")
df.head()

,Region,ProductLine,Month,Revenue,UnitsSold,DiscountPercent,Cost,Profit,ProfitMargin,KPI_01,KPI_02,KPI_03,KPI_04,KPI_05,KPI_06,KPI_07,KPI_08,KPI_09,KPI_10,KPI_11,KPI_12,KPI_13,KPI_14,KPI_15,KPI_16,KPI_17,KPI_18,KPI_19,KPI_20,KPI_21,KPI_22,KPI_23,KPI_24,KPI_25,KPI_26,KPI_27,KPI_28,KPI_29,KPI_30,KPI_31,KPI_32,KPI_33,KPI_34,KPI_35,KPI_36,KPI_37,KPI_38,KPI_39,KPI_40
0,East,Clothing,2023-01-01 00:00:00,181890,69,"0,28","122553,98","59336,02","0,33","104,4","100,26","84,07","99,84","115,72","108,62","98,34","115,8","113,63","70,08","106,3","107,12","72,85","114,91","93,97","105,16","111,21","87,6","118,52","79,26","111,74","108,97","127,39","73,58","131,22","84,08","120,64","77,37","76,65","63,46","100,97","86,57","118,95","95,2","104,3","116,46","105,26","82,91","117,61","90,59"
1,East,Clothing,2023-02-01 00:00:00,316208,952,"0,15","205913,08","110294,92","0,35","107,2","104,32","78,91","122,06","134,75","116,68","162,1","124,46","134,17","88,75","117,2","73,45","83,93","70,57","116,18","101,7","80,71","81,31","133,67","145,8","105,28","89,83","84,14","132,17","48,52","87,11","91,2","96,33","123,34","116,36","83,42","95,96","83,36","112,95","109,02","127,5","96,35","112,5","105,15","114,32"
2,East,Clothing,2023-04-01 00:00:00,483125,1021,"0,17","400566,74","82558,26","0,17","102,56","141,34","68,89","82,02","80,93","118,84","74,46","126,76","66,26","118,75",93,"108,58","77,16","124,77","106,34","106,54","58,53","103,76","71,63","119,35","65,62","83,06","141,55","104,86","110,39","116,79","131,92","116,83","105,27","106,16","112,17","99,13","71,95","106,23","71,27","108,21","121,33","101,07","88,5","75,74"
3,East,Clothing,2023-10-01 00:00:00,476960,1856,"0,13","311779,56","165180,44","0,35","57,08","116,4","145,13","104,72","55,09","139,26","100,34","126,38","124,98","69,35","96,43","102,58","107,8","111,6","94,55","97,87","99,93","66,22","127,34","89,94","92,07","114,52",66,"91,12","107,19","81,75","90,23","117,25","79,8","82,89","110,58","124,64","92,32","100,18","108,17","88,18","123,39","101,25","84,87","65,21"
4,East,Electronics,2023-04-01 00:00:00,432515,177,"0,02","301553,9","130961,1","0,3","128,72","104,47","99,22","103,45","92,77","98,1","111,19","82,11","123,05","124,37","94,86","81,17","128,59","86,49","97,7","76,43","113,44","87,3","126,34","146,01","87,44","88,7","100,16","89,44","97,98","112,66","92,87","72,72","148,59","121,93","92,89","90,96","109,71","90,64","97,44","81,56","89,29","113,26","124,89","96,62"


In [18]:
from sqlalchemy import create_engine

# Path to your SQLite database file
# database_file_path = "./data/test.db"
database_file_path = "./data/sales_db.db"

# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
engine = create_engine(f'sqlite:///{database_file_path}')
# file_url = "./data/all-states-history.csv"
# df = pd.read_csv(file_url).fillna(value = 0)
# df.to_sql(
#     'all_states_history',
#     con=engine,
#     if_exists='replace',
#     index=False
# )
df.to_sql(
    'sales_db',
    con=engine,
    if_exists='replace',
    index=False
)



100

In [19]:
MSSQL_AGENT_PREFIX = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [20]:
MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: 
SELECT TOP (10) [death]
FROM covidtracking 
WHERE state = 'TX' AND date LIKE '2020%'

Observation:
[(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state
is 'TX' and the date starts with '2020'. The query returned a list of tuples
with the number of deaths for each day in 2020. To answer the question,
I took the sum of all the deaths in the list, which is 27437.
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example

"""

In [21]:
# Create LangChain model
model2 = AzureChatOpenAI(
    openai_api_version=api_version,
    azure_deployment=deployment,
    azure_endpoint=endpoint,
    api_key=subscription_key,
    temperature = 0,
    max_tokens=500
)
db = SQLDatabase.from_uri(f'sqlite:///{database_file_path}')
toolkit = SQLDatabaseToolkit(db=db, llm=model2)

In [22]:
QUESTION = """How may patients were hospitalized during October 2020
in New York, and nationwide as the total of all states?
Use the hospitalizedIncrease column
"""

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=model2,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

In [23]:
output_2 = agent_executor_SQL.invoke(QUESTION)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: sales_dbI see there is only one table named "sales_db" which does not seem related to covid or hospitalization data. I don't see any table that would contain hospitalization data or a column named hospitalizedIncrease.

Thought: I don't have the relevant data to answer the question about hospitalized patients in October 2020 in New York and nationwide.

Final Answer: I don't know

> Finished chain.


In [24]:

# Or display it in markdown for better formatting
display(Markdown(output_2['input']))
display(Markdown(output_2['output']))

How may patients were hospitalized during October 2020
in New York, and nationwide as the total of all states?
Use the hospitalizedIncrease column


I don't know

In [25]:
QUESTION = """Which product line had the highest average profit margin in the West region during 2023, and how much higher was it compared to the lowest one?
"""

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=model2,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

In [26]:
output_challenge = agent_executor_SQL.invoke(QUESTION)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: 
""sales_dbAction: sql_db_schema
Action Input: sales_db
CREATE TABLE sales_db (
	"Region" TEXT, 
	"ProductLine" TEXT, 
	"Month" TEXT, 
	"Revenue" BIGINT, 
	"UnitsSold" BIGINT, 
	"DiscountPercent" TEXT, 
	"Cost" TEXT, 
	"Profit" TEXT, 
	"ProfitMargin" TEXT, 
	"KPI_01" TEXT, 
	"KPI_02" TEXT, 
	"KPI_03" TEXT, 
	"KPI_04" TEXT, 
	"KPI_05" TEXT, 
	"KPI_06" TEXT, 
	"KPI_07" TEXT, 
	"KPI_08" TEXT, 
	"KPI_09" TEXT, 
	"KPI_10" TEXT, 
	"KPI_11" TEXT, 
	"KPI_12" TEXT, 
	"KPI_13" TEXT, 
	"KPI_14" TEXT, 
	"KPI_15" TEXT, 
	"KPI_16" TEXT, 
	"KPI_17" TEXT, 
	"KPI_18" TEXT, 
	"KPI_19" TEXT, 
	"KPI_20" TEXT, 
	"KPI_21" TEXT, 
	"KPI_22" TEXT, 
	"KPI_23" TEXT, 
	"KPI_24" TEXT, 
	"KPI_25" TEXT, 
	"KPI_26" TEXT, 
	"KPI_27" TEXT, 
	"KPI_28" TEXT, 
	"KPI_29" TEXT, 
	"KPI_30" TEXT, 
	"KPI_31" TEXT, 
	"KPI_32" TEXT, 
	"KPI_33" TEXT, 
	"KPI_34" TEXT, 
	"KPI_35" TEXT, 
	"KPI_36" TEXT, 
	"KPI_37" TEXT, 
	"KPI_38" TEXT, 
	"KPI_39" 

In [27]:
display(Markdown(output_challenge['input']))
display(Markdown(output_challenge['output']))

Which product line had the highest average profit margin in the West region during 2023, and how much higher was it compared to the lowest one?


The product line "Toys" had the highest average profit margin in the West region during 2023, which was about 0.156 (15.6 percentage points) higher than the lowest one, "Clothing."

Explanation:
I queried the `sales_db` table for the average profit margin by product line in the West region for the year 2023. Since the `ProfitMargin` column was stored as text with commas as decimal separators, I replaced commas with dots and cast the values to float for accurate averaging. I grouped the results by `ProductLine` and ordered them by average profit margin descending to find the highest and lowest values. The SQL query used was:

```sql
SELECT ProductLine, AVG(CAST(REPLACE(ProfitMargin, ',', '.') AS FLOAT)) AS AvgProfitMargin
FROM sales_db
WHERE Region = 'West' AND Month LIKE '2023%'
GROUP BY ProductLine
ORDER BY AvgProfitMargin DESC
LIMIT 30
```

## STEP 4. Azure OpenAI Function Calling
What is the additional values
- provide specific isntructions for finding information 
- prioritzie queries for precise reulst and desired formats
- more control 

In [28]:

import json

## example of fucntion 
def get_current_weather(location, unit="Celsius"):
    """Get the current weather in a given location. 
    The default unit when not specified is Celsius"""
    if "merelbeke" in location.lower():
        return json.dumps(
            {"location": "Merelbeke", "country":"Belgium", "temperature": "20", "unit": unit}
        )
    elif "antwerpen" in location.lower():
        return json.dumps(
            {"location": "Antwerpen", "country":"Belgium", "temperature": "25", "unit": unit}
        )
    elif "las vegas" in location.lower():
        return json.dumps(
            {"location": "Las Vegas", "country":"USA", "temperature": "35", "unit": unit}
        )
    else:
        return json.dumps(
            {"location": location, "country":"unknown", "temperature": "unknown", "unit":  unit}
        )

get_current_weather("Merelbeke")

'{"location": "Merelbeke", "country": "Belgium", "temperature": "20", "unit": "Celsius"}'

In [29]:

# user prompt
weather_messages = [
    {"role": "user",
     "content": """What's the weather like in Merelbeke,
                   Antwerpen, and Las Vegass?"""
    }
]

#tool definition

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": """Get the current weather in a given
                              location.The default unit when not
                              specified is Celsius""",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": """The city and state,
                                        e.g. San Francisco, CA""",
                    },
                    "unit": {
                        "type": "string",
                        "default":"Celsius",
                        "enum": [ "Fahrenheit", "Celsius"],
                        "description": """The messuring unit for
                                          the temperature.
                                          If not explicitly specified
                                          the default unit is 
                                          Celsius"""
                    },
                },
                "required": ["location"],
            },
        },
    }
]

In [30]:
#defining the OpenAI clietn class
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)


#just call the class
response = client.chat.completions.create(
    model=v_model,
    messages=weather_messages,
    tools=tools,
    tool_choice="auto", 
)

response_message = response.choices[0].message
print ("Response message: \n" , response_message, "\n\n" )

tool_calls = response_message.tool_calls

print ("TOOLS CALLS: \n" , tool_calls , "\n\n" )

available_functions = {
    "get_current_weather": get_current_weather,
} 

answers = []

if tool_calls:
   
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit", "Celsius")  ## you can see from the function=Fuction that the only argument is location
        )
        answers.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  
    print ("Answers: \n" , answers , "\n\n" ) 
    
   
    def print_arguments(obj):
        "Recursively find and print JSON arguments or content with key/value pairs."
        if isinstance(obj, list):
            for item in obj:
                print_arguments(item)
        elif isinstance(obj, dict):
            # if it looks like a serialized JSON string of arguments/content, try parsing it
            for key, val in obj.items():
                if key in ("arguments", "content") and isinstance(val, str) and val.strip().startswith("{"):
                    try:
                        parsed = json.loads(val)
                        for k, v in parsed.items():
                            if k == "unit":
                                print(f"{k}: {v} \n")
                                v = 1
                            else:
                                print(f"{k}: {v}")
                    except Exception:
                        pass
                else:
                    print_arguments(val)

print_arguments(answers)

Response message: 
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_xRVBZaJaCFB2Z8vtyroRW16q', function=Function(arguments='{"location": "Merelbeke"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_6qUDCgwhWJPAw8iuMNnWg7Za', function=Function(arguments='{"location": "Antwerpen"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_zwYaZxgC3ToZ1PZndncYJoqc', function=Function(arguments='{"location": "Las Vegas"}', name='get_current_weather'), type='function')]) 


TOOLS CALLS: 
 [ChatCompletionMessageFunctionToolCall(id='call_xRVBZaJaCFB2Z8vtyroRW16q', function=Function(arguments='{"location": "Merelbeke"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_6qUDCgwhWJPAw8iuMNnWg7Za', function=Function(arguments='{"location": "Ant

In [77]:
import numpy as np
from sqlalchemy import text

def Total_KPI_01(reg, prodline):
    try:
        query = f"""
        SELECT region, productline,  KPI_01
        FROM sales_db
        WHERE region = '{reg}' AND 
              productline = '{prodline}'
        group by region, productline;
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
            
        if not result.empty:
            return result.to_dict('records')
        else:
            return np.nan
        return result
    except Exception as e:
        print(e)
        return np.nan
    
    
def sum_profit_cost (reg, prodline):
    try:
        query = f"""
        SELECT region, productline, sum(profit), sum(cost)
        FROM sales_db
        where region = '{reg}'  AND
            productline = '{prodline}'
        group by region, productline;
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
            
        if not result.empty:
            return result.to_dict('records')
        else:
            return np.nan
        return result
    except Exception as e:
        print(e)
        return np.nan

In [78]:
print(Total_KPI_01("East","Toys"))

print(sum_profit_cost("East","Toys"))

[{'Region': 'East', 'ProductLine': 'Toys', 'KPI_01': '63,61'}]
[{'Region': 'East', 'ProductLine': 'Toys', 'sum(profit)': 406321.0, 'sum(cost)': 860659.0}]


In [92]:
sql_messages = [
    {"role": "user",
     "content": """ How much what the summed up KPI_01 for Region East and productline Toys?"""
    },
    {"role": "user",
     "content": """ What is the sum of the profits and costs for Region East and productline Clothing"""
    },
    
]

In [95]:
tools_sql = [
    {
        "type": "function",
        "function": {
            "name": "get_total_kpi_01_for_region_productline",
            "description": """Retrieves the sum of the KPI_01 for a specified region and specified productline.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "reg": {
                        "type": "string",
                        "description": """The name of the region
                                          (e.g., 'East', 'West')."""
                    },
                    "prodline": {
                        "type": "string",
                        "description": """The name of the productline 
                                          (e.g. 'Toys','Clothing')."""
                    }
                },
                "required": ["reg", "prodline"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_sum_profit_cost_for_region_productline",
            "description": """Retrieves the sum of the profits and the costs for a specified region and specified productline
                                """,
            "parameters": {
                "type": "object",
                "properties": {
                    "reg": {
                        "type": "string",
                        "description": """The name of the region
                                          (e.g., 'East', 'West')."""
                    },
                    "prodline": {
                        "type": "string",
                        "description": """The name of the productline 
                                          (e.g. 'Toys','Clothing')."""
                    }
                },
                "required": ["reg", "prodline"]
            }
        }
    }
]

In [101]:
#just call the class
response = client.chat.completions.create(
    model=v_model,
    messages=sql_messages,
    tools=tools_sql,
    tool_choice="auto", 
)

response_sql_message = response.choices[0].message
tool_calls = response_sql_message.tool_calls

print (tool_calls)
sql_answers = []

available_functions = {
    "get_total_kpi_01_for_region_productline": Total_KPI_01,
    "get_sum_profit_cost_for_region_productline":sum_profit_cost
}  

if tool_calls:
   
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            reg=function_args.get("reg"),
            prodline=function_args.get("prodline"),
        )
        sql_answers.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": str(function_response),
            }
        ) 
    print ("Answers: \n" , sql_answers , "\n\n" ) 
    
import ast

for ans in sql_answers:
    content = ast.literal_eval(ans['content'])
    
    # handle both list and dict formats safely
    data = content[0] if isinstance(content, list) else content

    print(f"- Request: {ans['name']}")
    print(f"- Region: {data.get('Region', '')}")
    print(f"- ProdLine: {data.get('ProductLine', '')}")
    print(f"- KPI_01: {data.get('KPI_01', '').replace(',', '.')}")
    print(f"- Sum of profit: {data.get('sum(profit)', '')}")
    print(f"- Sum of Cost: {data.get('sum(cost)', '')}")

[ChatCompletionMessageFunctionToolCall(id='call_ZDdZ8yU4Cwcexsuwmn3xU766', function=Function(arguments='{"reg": "East", "prodline": "Toys"}', name='get_total_kpi_01_for_region_productline'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_WiMeJbFaUJqwOY1rBiTIanlu', function=Function(arguments='{"reg": "East", "prodline": "Clothing"}', name='get_sum_profit_cost_for_region_productline'), type='function')]
Answers: 
 [{'tool_call_id': 'call_ZDdZ8yU4Cwcexsuwmn3xU766', 'role': 'tool', 'name': 'get_total_kpi_01_for_region_productline', 'content': "[{'Region': 'East', 'ProductLine': 'Toys', 'KPI_01': '63,61'}]"}, {'tool_call_id': 'call_WiMeJbFaUJqwOY1rBiTIanlu', 'role': 'tool', 'name': 'get_sum_profit_cost_for_region_productline', 'content': "[{'Region': 'East', 'ProductLine': 'Clothing', 'sum(profit)': 417368.0, 'sum(cost)': 1040811.0}]"}] 


- Request: get_total_kpi_01_for_region_productline
- Region: East
- ProdLine: Toys
- KPI_01: 63.61
- Sum of profit: 
- Sum of Cost: 
-

In [97]:
second_response = client.chat.completions.create(
            model=v_model,
            messages=sql_messages,
        )
print (second_response)

ChatCompletion(id='chatcmpl-CWsHsAgRt3Mm2ocOP5CUIty0zauu5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Could you please provide the data or the table containing the KPIs, profits, and costs for the regions and product lines? This will help me calculate the sums you need.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1761951212, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier=None, system_fingerprint='fp_3dcd5944f5', usage=CompletionUsage(completion_tokens=36, prompt_tokens=44, total_tokens=80, completion_tokens_details=CompletionTokensDetails(accept